# Example notebook: File management

You can manage your SciServer files through SciServer Compute, just as you can through the SciServer Dashboard.

SciServer Compute can talk to other components of SciServer through a series of <em>modules</em>, one for each component. This example notebook shows how to use the <strong><code>SciServer.Files</code></strong> module to access SciServer's file system from within your Python scripts. Use the examples in this notebook to get started.

Before using this example notebook, make sure you have copied it out of the shared <strong><code>Getting Started</code></strong> volume and into one of your personal volumes. You can do this by running the <strong><code>Copy Example Notebooks.ipynb</code></strong> script in the same directory.

To run the example Python commands in this notebook, click in any of the Code cells below (the ones with the gray backgrounds). Click the play button at the top of the window (just below the menubar) to run the script, or press Shift-Enter. When you run a cell, its output of will appear directly below the cell.

## Import modules

Like any Python modules, the SciServer modules must be imported before being used. 

The next code block first imports the SciServer modules you will need for this example notebook, then imports some other required modules.

Edit the value of the variable <strong><code>Authentication_loginName</code></strong> to your username, so that the script will look in the right place for your files. 

Comments in the code block explain what each module does. To learn how to import other modules, see the Python 3.5 import documentation (https://docs.python.org/3.5/reference/import.html), or the documentation of the module(s) you are trying to import.

In [ ]:
import SciServer
from SciServer import Files # manage your file system space from SciServer Compute    #Authentication, Files
print('Imported SciServer modules')

from pprint import pprint # print human-readable output
print('Imported other needed modules')

print('\n')

# Enter your login name here to ensure that the script looks in your file system space
Authentication_loginName = 'PleaseEnterYourNameHere'

if (Authentication_loginName == 'PleaseEnterYourNameHere'):    
    raise Exception('Please enter your login name as the value for Authentication_loginName')
else:        
    print('Set value for Authentication_loginName: {0:}'.format(Authentication_loginName))


## Get help

At any point after the modules are imported, you can type "help (<em>name of module</em>)" to read the documentation for that module. This is true for all SciServer modules and most other modules as well. Try it below.

In [ ]:
help(Files)

# File Services

The basic unit of the SciServer file system is the File Service. Most SciServer users currently have access to only one file service, called FileServiceJHU.

## Show the file services that are available to you

The code cell below uses your login name to find the file services that are available to you.

Once you run this code cell, its output will display the name, description, and API endpoint of each file service you can access. Below this information, the output shows the full contents of each FileService object, using pprint to format its dictionary.

In [ ]:
# get available FileServices -- returns a list of dictionaries, one for each avialable file service
#   the for loop iterates through the list and prints information about each dictionary object

fileServicesList = Files.getFileServices()
for thisFileService in fileServicesList:
    print('name',':',thisFileService['name'])
    print('description',':',thisFileService['description'])
    print('apiEndpoint',':',thisFileService['apiEndpoint'])
    print('\n')
    print('The full dictionary:')
    pprint(thisFileService)

## Get file service information from a name

If you know the name of a file service, you can use <code>File.getFileServiceFromName(<em>name</em>)</code> command to get full information about that file service. Most SciServer users have access to only one file service, named <em>FileServiceJHU</em>.

In [ ]:
# get FileService from Name
fileServiceNames = Files.getFileServicesNames()
print('Names of available file services:')
pprint(fileServiceNames)

print('\n')
Files_FileServiceName = "FileServiceJHU"
print('File Service information for {0:}:'.format(Files_FileServiceName))
fileService = Files.getFileServiceFromName(Files_FileServiceName)
pprint(fileService)

## Get the API endpoint for a file service

Similarly, if you know the name of a file service, you can use the <code>File.__getFileServiceAPIUrl(<em>name</em>)</code> command to get its API endpoint. Most SciServer users have access to only one file service, named <em>FileServiceJHU</em>.

Don't forget that the command syntax requires two underscores before the letters.

In [ ]:
# get the API endpoint URL of a FileService
fileServiceAPIUrl = Files.__getFileServiceAPIUrl(fileService)
print(fileServiceAPIUrl)

## Show root volumes

Root volumes are the top level of the file system for each SciServer File Service. Most users will see <em>FileServiceJHU</em>, which has two root volumes:

<ul>
    <li><strong>Storage</strong>: Regularly backed up, but each user is limited to 10 GB.</li>
    <li><strong>Temporary</strong>: Unlimited but transient. See the <a href="http://www.sciserver.org/support/policies/">SciServer Compute Data Storage Policy</a> for more information.</li>
</ul>

Like everything in SciServer Compute, he names <em>Storage</em> and <em>Temporary</em> are case-sensitive.

In [ ]:
rootVolumes = Files.getRootVolumesInfo(fileService)
pprint(rootVolumes)

# User volumes, directories, and files

User volumes are shareable units of file space that you can create. They are SciServer equivalent of external drives, which can be mounted through Compute for yourself and others. User volumes can be created in either <em>Storage</em> or <em>Temporary</em>, and the appropriate storage policy applies to each user volume.

The code cell below prints information for each of the user volumes that you have access to.

In [ ]:
userVolumes = Files.getUserVolumesInfo(fileService)
pprint(userVolumes)

## Create a new user volume

The code cell below creates a new user volume in your SciServer file system space. Use the variable <code>Files_RootVolumeName1</code> to choose whether to store the file in Storage or Temporary, and name your new user volume with the variable <code>Files_VolumeName1</code>.

In [ ]:
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName1 = 'Storage'
Files_VolumeName1 = "created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.createUserVolume(fileService,"/".join([Files_RootVolumeName1,Authentication_loginName,Files_VolumeName1]))

print('Created new user volume: {0:}!'.format(Files_VolumeName1))

## Share one of your user volumes with another user

Once you create a user volume, you can share it with any other user on SciServer. The code cell below will share the new user volume you just created. If you know someone on SciServer you would like to share it with, enter their name for the variable <code>shareWithUser</code>.

In [ ]:
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeNameToShareFrom = 'Storage'
Files_VolumeNameToShare = "created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

shareWithUser = 'PleaseEnterSomeNameHere'

try:
    Files.shareUserVolume(fileService, 
    "/".join([Files_RootVolumeNameToShareFrom,Authentication_loginName,Files_VolumeNameToShare]), 
        sharedWith=shareWithUser, type="USER",allowedActions=["read"])
except:
    raise Exception('Please enter a valid login name for another SciServer user')

## Create a new directory

The code cell below creates a new directory within your <em>persistent</em> volume.

In [ ]:
# create a directory in the persistent volume
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName2 = "Storage"
Files_VolumeName2 = "persistent"
Files_NewDirectoryName2 = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)
Files.createDir(fileService, 
    "/".join([Files_RootVolumeName2,Authentication_loginName,Files_VolumeName2,Files_NewDirectoryName2]));

print('Created new user directory {0:} inside {1:} volume.'.format(Files_NewDirectoryName2, Files_VolumeName2))

## List contents of a directory

The code cell below creates a new directory within your <em>persistent</em> volume.

In [ ]:
# List content of remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName3 = "Storage"
Files_VolumeName3 = "persistent"
Files_DirectoryName3 = ""

fileService = Files.getFileServiceFromName(Files_FileServiceName)

dirList = Files.dirList(fileService, 
    "/".join([Files_RootVolumeName3,Authentication_loginName,Files_VolumeName3,Files_DirectoryName3]),
    level=4)
pprint(dirList)

## Upload a file

The code cell below creates a simple text file, then uploads it into the folder you just created in your <em>persistent</em> volume. If you prefer to upload your own file, change the value of <code>Files_LocalFileName</code>.

In [ ]:
# upload a text string into a file in the remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName4 = "Storage"
Files_VolumeName4 = "persistent"
Files_DirectoryName4 = "directory_created_through_example_notebook"

# This file gets created below, or replace with your own filename
Files_LocalFileName = "sharknado.csv"

# Create this simple file to upload
Files_LocalFileContent = "Year, Sharknado movies shown on U.S. television\n"
Files_LocalFileContent += "2012, 0\n"
Files_LocalFileContent += "2013, 1\n"
Files_LocalFileContent += "2014, 1\n"
Files_LocalFileContent += "2015, 1\n"
Files_LocalFileContent += "2016, 1\n"
Files_LocalFileContent += "2017, 1\n"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.upload(fileService, 
    "/".join([Files_RootVolumeName4,Authentication_loginName,Files_VolumeName4,Files_DirectoryName4,Files_LocalFileName]), 
             data=Files_LocalFileContent)

print('Uploaded file {0:} to {1:} volume in directory {2:}.'.format(Files_LocalFileName, Files_VolumeName4, Files_DirectoryName4))

## List contents of a directory

In [ ]:
# List content of remote directory
Files_FileServiceName = "FileServiceJHU"
Files_RootVolumeName3 = "Storage"
Files_VolumeName3 = "persistent"
Files_DirectoryName3 = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

dirList = Files.dirList(fileService, 
    "/".join([Files_RootVolumeName3,Authentication_loginName,Files_VolumeName3,Files_DirectoryName3]),
    level=4)
pprint(dirList)

## Copy a file from one place to another

In [ ]:
# upload a text string into a file in the remote directory
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeNameNow = "Storage"
Files_VolumeNameNow = "persistent"
Files_DirectoryNameNow = ""

Files_RootVolumeMoveTo = "Storage"
Files_VolumeNameMoveTo = "created_through_example_notebook"
Files_DirectoryNameMoveTo = ""

Files_RemoteFileName = "sharknado.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


# copy remote file into remote directory

Files.move(fileService, 
"/".join([Files_RootVolumeNameNow,Authentication_loginName,Files_VolumeNameNow,Files_DirectoryNameNow,Files_RemoteFileName]), 
           fileService, 
"/".join([Files_RootVolumeMoveTo,Authentication_loginName,Files_VolumeNameMoveTo,Files_DirectoryNameMoveTo,Files_RemoteFileName])
)

print('File copied')

## Download a file from SciServer

The downloaded file will appear in the directory you are currently in.

In [ ]:
# download a remote text file into a local directory
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDownloadFrom = "Storage"
Files_VolumeNameToDownloadFrom = "created_through_example_notebook"
Files_DirectoryNameToDownloadFrom = ""
Files_RemoteFileName = "sharknado.csv"
Files_LocalFileName = "sharknado_downloaded.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.download(fileService, 
    "/".join([Files_RootVolumeToDownloadFrom,Authentication_loginName,Files_VolumeNameToDownloadFrom,Files_DirectoryNameToDownloadFrom,Files_RemoteFileName]) 
    ,localFilePath=Files_LocalFileName)

print('Downloaded file, saved as {0:}'.format(Files_LocalFileName))

## Delete a file from SciServer file system

In [ ]:
# Delete remote file
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDeleteIn = "created_through_example_notebook"
Files_DirectoryNameToDeleteIn = ""
Files_RemoteFileName = "sharknado.csv"

fileService = Files.getFileServiceFromName(Files_FileServiceName)

Files.delete(fileService, 
"/".join([Files_RootVolumeToDeleteIn,Authentication_loginName,Files_VolumeNameToDeleteIn,Files_DirectoryNameToDeleteIn,Files_RemoteFileName]))

print('Deleted file {0:} from volume {1:}'.format(Files_RemoteFileName, Files_VolumeNameToDeleteIn))

## Delete a directory

In [ ]:
# delete folders
# Delete remote file
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDeleteIn = "persistent"
Files_DirectoryNameToDelete = "directory_created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


Files.delete(fileService, 
    "/".join([Files_RootVolumeToDeleteIn,Authentication_loginName,Files_VolumeNameToDeleteIn,Files_DirectoryNameToDelete]))

print('Deleted folder {0:} from volume {1:}'.format(Files_DirectoryNameToDelete, Files_VolumeNameToDeleteIn))

## Delete a user volume

In [ ]:
# delete user volumes
Files_FileServiceName = "FileServiceJHU"

Files_RootVolumeToDeleteIn = "Storage"
Files_VolumeNameToDelete = "created_through_example_notebook"

fileService = Files.getFileServiceFromName(Files_FileServiceName)


Files.deleteUserVolume(fileService, 
    "/".join([Files_RootVolumeToDeleteIn, Authentication_loginName, Files_VolumeNameToDelete]))

print('Deleted volume {0:}'.format(Files_VolumeNameToDelete))